Strava Cycling Stats

In [2]:
from stravaio import strava_oauth2
from stravaio import StravaIO
import pandas as pd
import numpy as np
import psycopg2
import datetime
import random
import time

TODO
 [ ] Add logging for all inserts made, use the athlete

In [26]:
class CylcingStats:
    def __init__(self, host="localhost", db="CyclingStats", usr="hank", pwd="soccer18"):
        # Postgres db conn info
        self.conn = None
        self.host = host,
        self.database = db,
        self.user = usr,
        self.password = pwd
        self.client = None
        
        # Inserting the Activities from the API to the DB
        self.insert_sql = 'INSERT INTO Activities(achievement_count, athlete, athlete_count, attribute_map, average_speed, average_watts, comment_count, commute, device_watts, discriminator, distance, elapsed_time, elev_high, elev_low, end_latlng, external_id, flagged, gear_id, has_kudoed, id, kilojoules, kudos_count, manual, map, max_speed, max_watts, moving_time, name, photo_count, private, start_date, start_date_local, start_latlng, swagger_types, timezone, total_elevation_gain, total_photo_count, trainer, type, upload_id, weighted_average_watts, workout_type) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);'
        self.insert_params = (None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None)
        
        # Getting the Activities' Ids from the DB to prevent dups
        self.select_sql_activity_id = 'SELECT id from activities;'
        self.db_activity_ids = None # returns a 2d tuple
        self.db_activity_ids_list = [] # the tuple is parsed to a 1d list
        
        # Activities dataframe
        self.cols_activities = ['achievement_count', 'athlete', 'athlete_count', 'attribute_map', 'average_speed', 'average_watts', 'comment_count', 'commute', 'device_watts', 'discriminator', 'distance', 'elapsed_time', 'elev_high', 'elev_low', 'end_latlng', 'external_id', 'flagged', 'gear_id', 'has_kudoed', 'id', 'kilojoules',
 'kudos_count', 'manual', 'map', 'max_speed', 'max_watts', 'moving_time', 'name', 'photo_count', 'private', 'start_date', 'start_date_local', 'start_latlng',
 'swagger_types', 'timezone', 'total_elevation_gain', 'total_photo_count', 'trainer', 'type', 'upload_id', 'weighted_average_watts',  'workout_type']
        
        # Strava API stuff
        # Keep track of calls, < 100/15mins & < 1,000/day
        # TODO: going to need to store this stuff in the db to make sure that the user doesnt go over and lose license
        self.api_calls = 0
        self.access_token = 'b584ab27fdef203d27f7da87df7a930902484fed' # TODO: this should be checked + refreshed auto
        
        self.athlete_id = 49039115 # TODO: this should be updated auto
        self.athlete = None #JSON for strava athlete
        
        self.activities = [] # List of JSON for strava athletes activities
        self.activities_df = None # Activities dataframe
        self.activities_old = [] # List of Activities that are already in the db
        self.activities_new = [] # List of Activities that are not in the db
        
        self.streams = [] # Stream obj with to_dict which can be used to make a pandas df
        self.streams_df = None # Streams dataframe, only ones that arent in the db
        
        self.verbose = True # for terminal logging (db logging is handled via the db)
        
        
    #######################################################################################
    # Master Methods
    #######################################################################################
    ## These just provide a one-click to do anything instead of having to call multiple methods
    ## The only methods that should really be called after its stable
    # Method that will ultimately try to do the full suite of connections, updates, and inserts
    def OneClick(self):
        self.ConnectStravaApi()
        self.GetActivities()
        self.ParseActivities()
        self.GetDbActivityIds()
        self.ParseDbActivityIds()
        self.InsertActivities()
    
    #######################################################################################
    # Connections
    #######################################################################################
    
    def RefreshAccessToken(self):
        pass# TODO
    
    def ConnectStravaApi(self):
        self.client = StravaIO(access_token = self.access_token) 

    # Don't use this until figure out how to pass vars into the params
    def ConnectDb(self):
        self.conn = psycopg2.connect(host=self.host, database=self.database,
                                      user=self.user, password=self.password)

    def CloseDb(self):
        self.conn.close()

    #######################################################################################
    # Activities
    #######################################################################################
    
    # Pull Activity Ids from db to make sure dups arent inserted
    def GetDbActivityIds(self):
        self.conn = psycopg2.connect(host = "localhost", database = 'CyclingStats',
                      user = 'postgres', password = 'soccer18')
        with self.conn:
            with self.conn.cursor() as curs:
                curs.execute(self.select_sql_activity_id)
                self.db_activity_ids = curs.fetchall()
    
    # These are returned as a tuple, parsing them into a 1d list to compare against db Activity Ids 
    #to make sure dups aren't inserted
    def ParseDbActivityIds(self): 
        for rec in self.db_activity_ids:
             self.db_activity_ids_list.append(rec[0])
   
    # Populate activities_old, meaning activities that are already in the db
    # *** the activities df has to have already been built before calling this
    def IdentifyOldActivities(self):
        self.conn = psycopg2.connect(host = "localhost", database = 'CyclingStats',
                              user = 'postgres', password = 'soccer18')
        with self.conn:
            with self.conn.cursor() as curs:
                for record in self.activities_df.itertuples():
                    # Make sure the Activity isnt already in the db
                    if record.id not in self.db_activity_ids_list:
        
    # Insert activities from the Strava API into the Postgres CyclingStats db
    def InsertActivities(self):
        self.conn = psycopg2.connect(host = "localhost", database = 'CyclingStats',
                              user = 'postgres', password = 'soccer18')
        with self.conn:
            with self.conn.cursor() as curs:
                # TODO: need to pull this into separate method and make the ids a list since streams using same logic
                # to prevent excessive api calls
                
                # TODO: need to make sure at max __ records to not go over api limit
                for record in self.activities_df.itertuples():
                    # Make sure the Activity isnt already in the db
                    if record.id not in self.db_activity_ids_list:
                        self.insert_params = (record.achievement_count, str(record.athlete), record.athlete_count, str(record.attribute_map), record.average_speed, record.average_watts, record.comment_count, record.commute, str(record.device_watts), str(record.discriminator), record.distance, record.elapsed_time, record.elev_high, record.elev_low, str(record.end_latlng), str(record.external_id), record.flagged, str(record.gear_id), record.has_kudoed, record.id, record.kilojoules, record.kudos_count, record.manual, str(record.map), record.max_speed, record.max_watts, record.moving_time, str(record.name), record.photo_count, record.private, record.start_date, record.start_date_local, str(record.start_latlng), str(record.swagger_types), str(record.timezone), record.total_elevation_gain, record.total_photo_count, record.trainer, str(record.type), record.upload_id, record.weighted_average_watts, record.workout_type)
                        curs.execute(self.insert_sql, self.insert_params)
                        print('Inserted Activity {0} successfully'.format(str(record.id)))
                        self.insert_params = (None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None)
                    else:
                        print('Activity {0} already in the database'.format(str(record.id)))
        
        self.CloseDb()
        
    # Pulls a list of activities for the current athlete from the Strava API
    def GetActivities(self):
        self.list_activities = self.client.get_logged_in_athlete_activities()
    
    # Parses the list of activities into a pandas dataframe
    # Obviously call GetActivities prior
    def ParseActivities(self):
        # Parse out each activities and store each record in a list
        for obj in self.list_activities:
            record = [obj.achievement_count, obj.athlete, obj.athlete_count, obj.attribute_map, obj.average_speed, obj.average_watts, obj.comment_count, obj.commute, obj.device_watts, obj.discriminator, obj.distance, obj.elapsed_time, obj.elev_high, obj.elev_low, obj.end_latlng, obj.external_id, obj.flagged, obj.gear_id, obj.has_kudoed, obj.id, obj.kilojoules, obj.kudos_count, obj.manual, obj.map, obj.max_speed, obj.max_watts, obj.moving_time, obj.name, obj.photo_count, obj.private, obj.start_date, obj.start_date_local, obj.start_latlng, obj.swagger_types, obj.timezone, obj.total_elevation_gain, obj.total_photo_count, obj.trainer, obj.type, obj.upload_id, obj.weighted_average_watts, obj.workout_type]
 
            self.activities.append(record)
            
        # Build a dataframe from the activities
        self.activities_df = pd.DataFrame(self.activities, columns=self.cols_activities)
        
    #######################################################################################
    # Activity Streams
    #######################################################################################
    
    # Pull a list of Streams based on activities_df.id
    def GetStreams(self):
        self.stream = self.client.get_activity_streams(5434486134, athlete_id = self.athlete_id)
      
    #######################################################################################
    # UI 
    #######################################################################################
    # Will be called by a button on the UI for Users to export out a listing of their activities by date
    #def ExportActivitiesFromDb(self, start_date, end_date):
        # Pull list of activities from db filtered on date
        
        # Store as dataframe
        
        # Export dataframe

In [27]:
cs = CylcingStats()

In [28]:
cs.OneClick()

Fetched 30, the latests is on 2020-03-09 16:59:31+00:00
Fetched 30, the latests is on 2020-06-12 16:11:46+00:00
Fetched 30, the latests is on 2020-07-25 17:05:59+00:00
Fetched 30, the latests is on 2020-12-20 16:53:14+00:00
Fetched 30, the latests is on 2021-04-29 21:01:32+00:00
Fetched 9, the latests is on 2021-06-08 11:22:52+00:00
Activity 2941957268 already in the database
Activity 2945778772 already in the database
Activity 2947002095 already in the database
Activity 2953864663 already in the database
Activity 2955608972 already in the database
Activity 2958821008 already in the database
Activity 2961046872 already in the database
Activity 2963511616 already in the database
Activity 2969146363 already in the database
Activity 2973616333 already in the database
Activity 2976952200 already in the database
Activity 2988755347 already in the database
Activity 2991493125 already in the database
Activity 2994420097 already in the database
Activity 2999995400 already in the database
Activ

In [31]:
cs.GetStreams()

In [34]:
cs.stream

Streams for 5434486134
Keys: ['time', 'distance', 'altitude', 'velocity_smooth', 'heartrate', 'cadence', 'watts', 'moving', 'grade_smooth', 'lat', 'lng']
Access: obj.key or obj.to_dict() to load into a pd.DataFrame()

In [38]:
a = cs.stream.to_dict()

In [40]:
b = pd.DataFrame(a)

In [42]:
b['activity_id'] = 5434486134

In [47]:
b

,time,distance,altitude,velocity_smooth,heartrate,cadence,watts,moving,grade_smooth,lat,lng,activity_id
0,0,1.9,126.6,0.0,85,1,125,False,0.0,53.988715,-1.540488,5434486134
1,1,4.1,126.6,0.0,85,39,119,True,0.0,53.988732,-1.540498,5434486134
2,2,6.8,126.6,2.5,85,49,106,True,0.0,53.988753,-1.540510,5434486134
3,3,9.9,126.6,2.7,86,52,94,True,0.0,53.988780,-1.540523,5434486134
4,4,13.3,126.6,2.9,86,52,94,True,-1.4,53.988807,-1.540540,5434486134
...,...,...,...,...,...,...,...,...,...,...,...,...
4840,4995,28184.7,118.8,9.0,119,0,0,True,-2.2,53.992664,-1.542407,5434486134
4841,4996,28193.8,118.6,9.0,120,0,0,True,-2.2,53.992744,-1.542439,5434486134
4842,4997,28202.9,118.4,9.0,120,0,0,True,-1.7,53.992819,-1.542472,5434486134
4843,4998,28211.9,118.2,9.0,120,0,0,True,-1.5,53.992900,-1.542506,5434486134
